In [4]:
from utils import *
import os
import pandas as pd

In [8]:
df = pd.read_csv('arxiv.csv')
df.shape

/var/folders/wg/cdt_cw_5265_z_gwnxlf0tv80000gn/T/ipykernel_48419/3535194879.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('arxiv.csv')


(327288, 14)

In [9]:
df['abstract'].values

array(["  We discuss the results from the combined IRAC and MIPS c2d Spitzer Legacy\nobservations of the Serpens star-forming region. In particular we present a set\nof criteria for isolating bona fide young stellar objects, YSO's, from the\nextensive background contamination by extra-galactic objects. We then discuss\nthe properties of the resulting high confidence set of YSO's. We find 235 such\nobjects in the 0.85 deg^2 field that was covered with both IRAC and MIPS. An\nadditional set of 51 lower confidence YSO's outside this area is identified\nfrom the MIPS data combined with 2MASS photometry. We describe two sets of\nresults, color-color diagrams to compare our observed source properties with\nthose of theoretical models for star/disk/envelope systems and our own modeling\nof the subset of our objects that appear to be star+disks. These objects\nexhibit a very wide range of disk properties, from many that can be fit with\nactively accreting disks to some with both passive disks 

In [15]:
# Read in extracted_large.json and print how many hypotheses were extracted:
import json

with open('extracted_large.json', 'r') as f:
    hypotheses = json.load(f)

print(f"Number of hypotheses extracted: {len(hypotheses)}")

Number of hypotheses extracted: 89


## Generation

In [69]:
# Critic Chain: Takes a hypothesis and provides a critique.
critic_template = """
Given the following hypothesis:
{hypothesis}

Analyze each part of the hypothesis:
1. Clarity: Are the statements clear and easy to understand?
2. Coherence: Do the different parts of the hypothesis logically flow together? Does it seem like the proposed solution would work?
3. Scientific Validity: Are there any scientific inaccuracies or assumptions that seem unfounded?

After your analysis, provide specific feedback on how each field (Problem, Solution, Methodology, Evaluation, Results) can be improved.
"""
critic_prompt = PromptTemplate(input_variables=["hypothesis"], template=critic_template)
critic_chain = LLMChain(llm=llm, prompt=critic_prompt, output_key="critique")

# Reviser Chain: Takes original hypothesis and critique, then provides a revised hypothesis.
reviser_template = """
Given the original hypothesis:
{hypothesis}

And based on the critique provided:
{critique}

Revise the hypothesis by addressing each point of the critique. Ensure the new version is clearer, more coherent, and scientifically valid.
"""
reviser_prompt = PromptTemplate(input_variables=["hypothesis", "critique"], template=reviser_template)
reviser_chain = LLMChain(llm=llm, prompt=reviser_prompt, output_key="revised_hypothesis")

# Overall Sequential Chain: Critiques the hypothesis and then revises it.
overall_chain = SequentialChain(
    chains=[critic_chain, reviser_chain],
    input_variables=["hypothesis"],
    output_variables=["critique", "revised_hypothesis"],
    verbose=True)

# Test the chain with a provided hypothesis
hypothesis = {
    "Problem": "Issues in determining basic characteristics of black holes and their surrounding disks in X-ray binary, using models of the source's disk X-ray continuum. A key issue is the determination of the \"color correction factor\".",
    "Solution": "Using observational data to estimate the color correction factor by modeling the disk spectrum with saturated Compton scattering.",
    "Methodology": "The work is based on two observations made by XMM-Newton on GX 339-4. These observations offer high-quality data at low energies. The spectra were then fitted to these models.",
    "Evaluation": "The quality of fit of the spectra to the models was examined. Other models were also tested for fit.",
    "Results": "The spectra fits well with the model and provides reasonable values for the color correction factor. However, the high-soft-state continuum cannot be adequately fitted by the latest disk models."
}
result = overall_chain({"hypothesis": str(hypothesis)})




> Entering new SequentialChain chain...

> Finished chain.


In [70]:
print(result['critique'])

Analysis:

1. Clarity: The hypothesis is relatively clear but contains technical terms that may be difficult for non-experts to understand. For example, terms like "X-ray binary", "color correction factor", "saturated Compton scattering", and "high-soft-state continuum" may require more explanation or context.
   
2. Coherence: The different parts of the hypothesis logically flow together. The problem is clearly stated, a solution is proposed, the methodology is explained, an evaluation process is detailed, and results are provided. 

3. Scientific Validity: The hypothesis appears to be scientifically valid. It uses real-world observational data and incorporates established scientific models. However, the validity of the results would depend on the accuracy and reliability of the data and models used.

Feedback:

Problem: The problem statement could be improved by providing more background information to help non-experts understand the issue. For example, briefly explaining what a blac

In [71]:
print(result['revised_hypothesis'])

{'Problem': 'The study aims to determine basic attributes of black holes and their surrounding disks, specifically in X-ray binary systems. In simple terms, X-ray binaries are systems where a black hole and a star are in close proximity, and the intense gravitational pull of the black hole causes matter from the star to form a disk around the black hole. The main challenge is the determination of the "color correction factor", a numerical factor that helps us to understand the temperature and energy of the emitting source.', 

'Solution': 'The proposed solution is to estimate the color correction factor using observational data. This is achieved by modeling the spectrum of the disk around the black hole with a phenomenon known as saturated Compton scattering, where photons gain energy by interacting with high-energy particles.', 

'Methodology': 'The research utilized two observational datasets provided by the XMM-Newton space telescope on the X-ray binary system GX 339-4. The high-qua

In [72]:
import ast

hypothesis_dict = ast.literal_eval(result['revised_hypothesis'])

In [1]:
import ast
import yaml
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
config = yaml.safe_load(open("../config.yml"))
API_KEY = config['api_key']
DEPLOYMENT_NAME = config['deployment_name']
BASE_URL = config['base_url']
API_VERSION = config['api_version']

llm = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version=API_VERSION,
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
)

# Critic Chain: Takes a hypothesis and provides a critique.
critic_template = """
Given the following hypothesis:
{hypothesis}

Analyze each part of the hypothesis:
1. Clarity: Are the statements clear and easy to understand?
2. Coherence: Do the different parts of the hypothesis logically flow together? Does it seem like the proposed solution would work?
3. Scientific Validity: Are there any scientific inaccuracies or assumptions that seem unfounded?

After your analysis, provide specific feedback on how each field (Problem, Solution, Methodology, Evaluation, Results) can be improved.
Specifically, give feedback on the frailties of the idea as a whole, and suggest potential enhancements.
"""

# Reviser Chain: Takes original hypothesis and critique, then provides a revised hypothesis.
reviser_template = """
Given the original hypothesis:
{hypothesis}

And based on the critique provided:
{critique}

Revise the hypothesis by addressing each point of the critique. Ensure the new version is clearer, more coherent, and scientifically valid.
"""

def adversarial_update_hypothesis(hypothesis):
    critic_prompt = PromptTemplate(input_variables=["hypothesis"], template=critic_template)
    critic_chain = LLMChain(llm=llm, prompt=critic_prompt, output_key="critique")
    reviser_prompt = PromptTemplate(input_variables=["hypothesis", "critique"], template=reviser_template)
    reviser_chain = LLMChain(llm=llm, prompt=reviser_prompt, output_key="revised_hypothesis")
    overall_chain = SequentialChain(
        chains=[critic_chain, reviser_chain],
        input_variables=["hypothesis"],
        output_variables=["critique", "revised_hypothesis"],
        verbose=False)
    result = overall_chain({"hypothesis": str(hypothesis)})
    return result

In [2]:
import re
from preprocessing import Hypothesis
from tqdm.notebook import tqdm

def improve_hypothesis(hypothesis: str, n_iters: int = 3) -> str:
    for i in tqdm(range(n_iters)):
        revised_hypothesis = adversarial_update_hypothesis(hypothesis)
        hyp_str = revised_hypothesis['revised_hypothesis']
        hyp_str = hyp_str.replace('\n', ' ')
        # Remove everything before the first { and after the last }
        hyp_str = re.sub(r'^.*?\{', '{', hyp_str)
        hypothesis = hyp_str
    return hypothesis

hypothesis_to_improve = {
    "Problem": "Issues in determining basic characteristics of black holes and their surrounding disks in X-ray binary, using models of the source's disk X-ray continuum. A key issue is the determination of the \"color correction factor\".",
    "Solution": "Using observational data to estimate the color correction factor by modeling the disk spectrum with saturated Compton scattering.",
    "Methodology": "The work is based on two observations made by XMM-Newton on GX 339-4. These observations offer high-quality data at low energies. The spectra were then fitted to these models.",
    "Evaluation": "The quality of fit of the spectra to the models was examined. Other models were also tested for fit.",
    "Results": "The spectra fits well with the model and provides reasonable values for the color correction factor. However, the high-soft-state continuum cannot be adequately fitted by the latest disk models."
}

improve_hypothesis(hypothesis=hypothesis_to_improve, n_iters=3)

  0%|          | 0/3 [00:00<?, ?it/s]

'{\'Problem\': \'We are tasked with the challenge of accurately determining the fundamental features of black holes and the disks that surround them in X-ray binary, a type of binary star system that emits X-rays. This process is pivotal to enhancing our understanding of how matter behaves in extreme gravitational fields and the nature of black holes. The importance of achieving this lies in its potential to contribute significantly to the broader field of astrophysics, enabling us to understand the universe better. The critical part of this process is the determination of the "color correction factor", a measure of how the color of an object appears to change due to relativistic effects when it is moving at speeds close to the speed of light.\',   \'Solution\': \'We propose a solution that utilizes observational data to derive estimates for the color correction factor. This involves modeling the disk spectrum through the innovative concept of "saturated Compton scattering". In this pr

In [11]:
import pandas as pd

df = pd.read_csv("../data/processed/extracted_hypotheses.csv")
df

,Problem,Solution
0,How can young stellar objects (YSOs) in the Se...,The authors propose a set of criteria for isol...
1,How does the Intermediate Polar (IP) EX Hya be...,Spectroscopic observations and spin Doppler to...
2,"The nature, structure, and dynamics of the sol...",Observations of the chromosphere at millimeter...
3,How can we theoretically understand and model ...,A theoretical framework is proposed that invol...
4,How can single source data sets from the Mock ...,"An end-to-end pipeline was implemented, consis..."
...,...,...
86,What are the characteristics and potential cla...,Using high-resolution near-infrared spectrosco...
87,What is the origin of the Milky Way's globular...,These unusual globular clusters are distinct f...
88,How can the missing satellite problem and the ...,"Axinos, generated by the decay of the next-to-..."
89,What are the properties of Intermediate Polars...,The study uses phase-resolved spectroscopy and...


In [12]:
print("Problem: "+df.iloc[0]['Problem'])
print("Solution: "+df.iloc[0]['Solution'])

Problem: How can young stellar objects (YSOs) in the Serpens star-forming region be accurately identified and distinguished from extra-galactic background contamination?
Solution: The authors propose a set of criteria for isolating YSOs from the background contamination. They also use color-color diagrams to compare observed source properties with theoretical models for star/disk/envelope systems, and model a subset of objects that appear to be star+disks. They conduct a spatial clustering analysis to understand the distribution of YSOs and the background extra-galactic population.


In [13]:
len(df)

91